In [81]:
%matplotlib inline

import json
import sys
import re
import codecs
import requests
import urllib

# Topic 2: Collecting Social Media Data

This notebook contains examples for using web-based APIs (Application Programmer Interfaces) to download data from social media platforms.
Our examples will include:

- Reddit
- Facebook
- Twitter

For most services, we need to register with the platform in order to use their API.
Instructions for the registration processes are outlined in each specific section below.

We will use APIs because they *can* be much faster than manually copying and pasting data from the web site, APIs provide uniform methods for accessing resources (searching for keywords, places, or dates), and it should conform to the platform's terms of service (important for partnering and publications).
Note however that each of these platforms has strict limits on access times: e.g., requests per hour, search history depth, maximum number of items returned per request, and similar.

<hr>
<img src="files/RedditLogo.jpg" width="20%">

## Topic 2.1: Reddit API

Reddit's API is probably the easiest to use and does not require credentials to access data on its subreddit pages.
Instead, one need only provide a semi-unique User-Agent string that identifies the application or purpose.
I'm not sure what will happen if the user-agent is duplicated, but I don't think it would be a big deal other than possible blocks or loss of access if a particularly malicious user uses the same user-agent string.

In [131]:
# For our first piece of code, we need to import the package 
# that connects to Reddit. Praw is a thin wrapper around reddit's 
# web APIs and works well

import praw

In [133]:
# Now we specify a "unique" user agent for our code
# This is primarily for identification, I think, and some
# user-agents of bad actors might be blocked
redditApi = praw.Reddit(user_agent='crisis_informatics_v01')

### Capturing Reddit Posts

Now for a given subreddit, we can get the newest posts to that sub. 
Post titles are generally short, so you could treat them as something similar to a tweet.

In [136]:
subreddit = "worldnews"

submissions = redditApi.get_subreddit(subreddit).get_new(limit=10)
for post in submissions:
    print post.title

My Shiba and Her Pup Just Had a "Moment"
My scruffy looking nerf herder turned 1... Treats for everybody!
Our Daughter's new friend
French strikers to shut nuclear power plant in bid to see labour reforms scrapped as country forced to dip into strategic fuel reserves
On a nice sunny day he likes to get behind these bushes, prop his head up on a forked branch and enjoy the the heat
Italy receives 2,600 migrants in just 24 hours
State Dept. watchdog: Clinton violated email rules
Putin Releases Ukrainian Pilot After 2 Years In Prison
Taliban Name Lesser-Known Cleric as Their New Leader
How i got woken up this morning. Thanks oso..


### Leveraging Reddit's Voting

Getting the new posts gives us the most up-to-date information. 
You can also get the "hot" posts, "top" posts, etc. that should be of higher quality. 
In theory.
__Caveat emptor__

In [135]:
subreddit = "worldnews"

submissions = redditApi.get_subreddit(subreddit).get_hot(limit=5)
for post in submissions:
    print post.title

Adidas shoe manufacture returns to Germany, fully automated instead of relying on human labour in Asia
Tony Blair: Britain and US ‘profoundly’ underestimated chaos brought about by toppling of Saddam Hussein | People | News
South Africa Just Lifted Its Ban on the Rhino Horn Trade
Poland starts logging primeval Bialowieza forest despite protests: More than 180,000 cubic metres of forest to be cut down in area that is home to Europe’s largest mammal and tallest trees
Grizzlies & polar bears are now mating - "I hate to say it, but from a genetic perspective, it’s quite likely grizzly bears will eat polar bears up, genetically." Warming Arctic allowing the 2 species to come into contact more often.


### Following Multiple Subreddits

Reddit has a mechanism called "multireddits" that essentially allow you to view multiple reddits together as though they were one.
To do this, you need to concatenate your subreddits of interesting using the "+" sign.

In [153]:
subreddit = "worldnews+aww"

submissions = redditApi.get_subreddit(subreddit).get_new(limit=10)
for post in submissions:
    print post.title

Bunny Will Win Your Heart
Professor: Fighting Global Warming Will Impoverish Everyone
The eggs under my balcony hatched!
For my cake day I thought I would share my kitty sherlock, who looks like he will kill me if I turn heater off!
Found this little guy on the side of the road this morning. Man he sure is loud!
When life hands you a bundle of baby St. Bernards...
Aow rackün.... rackün, stahp!
Brother and Sister
Koala Konga!
During the second day of U.S. President Barack Obama’s three-day visit to Vietnam, Obama continued to push for passage of the Trans-Pacific Partnership trade deal


### Accessing Reddit Comments

While you're never supposed to read the comments, for certain live streams or new and rising posts, the comments may provide useful insight into events on the ground or people's sentiment.
New posts may not have comments yet though.

Comments are attached to the post title, so for a given submission, you can pull its comments directly.

Note Reddit returns pages of comments to prevent server overload, so you will not get all comments at once and will have to write code for getting more comments than the top ones returned at first.
This pagination is performed using the MoreXYZ objects (e.g., MoreComments or MorePosts).

In [152]:
subreddit = "worldnews"

breadthCommentCount = 2

submissions = redditApi.get_subreddit(subreddit).get_hot(limit=1)
for post in submissions:
    print post.title
    
    # Get the top few comments
    for comment in post.comments[:breadthCommentCount]:
        if isinstance(comment, praw.objects.MoreComments):
            continue
        
        print "---", comment.name, "---"
        print "\t", comment.body
        
        for reply in comment.replies[:breadthCommentCount]:
            if isinstance(reply, praw.objects.MoreComments):
                continue
            
            print "\t", "---", reply.name, "---"
            print "\t\t", reply.body

Adidas shoe manufacture returns to Germany, fully automated instead of relying on human labour in Asia
--- t1_d3iz783 ---
	Fast forward 20 years, and they go back to Asia for cheaper maintenance of autofabs.
	--- t1_d3izrjd ---
		You can ship auto fab shops, you cannot ship people for labour (anymore). 
	--- t1_d3j0jrw ---
		In 20 years robots will be fixing robots and we will be laying around watching "ow my balls" all day long. 
--- t1_d3iw3rx ---
	This is the best tl;dr I could make, [original](http://www.theguardian.com/world/2016/may/25/adidas-to-sell-robot-made-shoes-from-2017) reduced by 65%. (I'm a bot)
*****
> More than 20 years after Adidas ceased production activities in Germany and moved them to Asia, chief executive Herbert Hainer unveiled to the press the group&#039;s new prototype &quot;Speedfactory&quot; in Ansbach, southern Germany.

> The 4,600-square-metre plant is still being built but Adidas opened it to the press, pledging to automate shoe production - which is cu

### Other Functionality

Reddit has a deep comment structure, and the code above only goes two levels down (top comment and top comment reply). 
You can view Praw's additional functionality, replete with examples on its website here: http://praw.readthedocs.io/

<hr>
<img src="files/FacebookLogo.jpg" width="20%">

## Topic 2.2: Facebook API

Getting access to Facebook's API is slightly easier than Twitter's in that you can go to the Graph API explorer, grab an access token, and immediately start playing around with the API.
The access token isn't good forever though, so if you plan on doing long-term analysis or data capture, you'll need to go the full OAuth route and generate tokens using the approved paths.

In [9]:
# As before, the first thing we do is import the Facebook
# wrapper

import facebook

### Connecting to the Facebook Graph

Facebook has a "Graph API" that lets you explore its social graph. 
For privacy concerns, however, Facebook's Graph API is extremely limited in the kinds of data it can view.
For instance, Graph API applications can now only view profiles of people who already have installed that particular application.
These restrictions make it quite difficult to see a lot of Facebook's data.

That being said, Facebook does have many popular public pages (e.g., BBC World News), and articles or messages posted by these public pages are accessible.
In addition, many posts and comments made in reply to these public posts are also publically available for us to explore.

To connect to Facebook's API though, we need an access token (unlike Reddit's API).
Fortunately, for research and testing purposes, getting an access token is very easy.

#### Acquiring a Facebook Access Token

1. Log in to your Facebook account
1. Go to Facebook's Graph Explorer (https://developers.facebook.com/tools/explorer/)
1. Copy the *long* string out of "Access Token" box and paste it in the code cell bedlow

<img src="files/FacebookInstructions_f1.png"/>

In [154]:
fbAccessToken = "EAACEdEose0cBAPHZCVEKwbUFbObQpqxXjsUGMnypoma9r2IJseYawZAkzm6HNqtJRpvFAxspIDC2yHHqJEvv63OvXAWBGP3vfIqxxIpoO5ZBgb0Gip9ZAx2XdcfI12QlkYM2kGNl4YHMonakAELw8Br8RDRCOXXJZATwJEratPAZDZD"

Now we can use the Facebook Graph API with this temporary access token (it does expire after maybe 15 minutes).

In [156]:
# Connect to the graph API, note we use version 2.5
graph = facebook.GraphAPI(access_token=fbAccessToken, version='2.5')

### Parsing Posts from a Public Page

To get a public page's posts, all you need is the name of the page. 
Then we can pull the page's feed, and for each post on the page, we can pull its comments and the name of the comment's author.
While it's unlikely that we can get more user information than that, author name and sentiment or text analytics can give insight into bursting topics and demographics.

In [165]:
# What page to look at?
targetPage = "nytimes"

# Other options for pages:
# nytimes, bbc, bbcamerica, bbcafrica, redcross, disaster

maxPosts = 10 # How many posts should we pull?
maxComments = 5 # How many comments for each post?

post = graph.get_object(id=targetPage + '/feed')

# For each post, print its message content and its ID
for v in post["data"][:maxPosts]:
    print "---"
    print v["message"], v["id"]
        
    # For each comment on this post, print its number, 
    # the name of the author, and the message content
    print "Comments:"
    comments = graph.get_object(id='%s/comments' % v["id"])
    for (i, comment) in enumerate(comments["data"][:maxComments]):
        print "\t", i, comment["from"]["name"], comment["message"]


---
Q&A: Pamela Paul, editor of The New York Times Book Review, is here to answer your questions about how books get reviewed. 

Ever wondered how we choose the books or where we store all the galleys? Ask your questions for Pamela in the comments. 5281959998_10150815787194999
Comments:
	0 John Geremia SHillary politicized dead first graders but excused herself for millions of dead Iraqis and thousands of dead and injured American veterans. Said, guns were coming into New York from Vermont.....desperate liar.........

 She was wrong on NAFTA, TPP, Minimum $12 wage, Marriage equality, using a private server, voting on the Iraq War and the Patriot Act, and The Panama Trade Agreement. What makes her a good candidate? I was at the Nevada convention. Nothing was thrown aNd there was no violence.
	1 Mohammed Junaid Good review keep it up
	2 Abel Pam Let's get to work!
	3 Sarabh Rai 😎😎😎
	4 Annie Medina Hello
---
“As a child, my classmates would make fun of me by calling me a redskin. I’m sure

<hr>
<img src="files/TwitterLogo.png" width="20%">

## Topic 2.1: Twitter API

Twitter's API is probably the most useful and flexible but takes several steps to configure. 
To get access to the API, you first need to have a Twitter account and have a mobile phone number (or any number that can receive text messages) attached to that account.
Then, we'll use Twitter's developer portal to create an "app" that will then give us the keys tokens and keys (essentially IDs and passwords) we will need to connect to the API.

So, in summary, the general steps are:

0. Have a Twitter account,
1. Configure your Twitter account with your mobile number,
2. Create an app on Twitter's developer site, and
3. Generate consumer and access keys and secrets.

We will then plug these four strings into the code below.

In [82]:
# For our first piece of code, we need to import the package 
# that connects to Twitter. Tweepy is a popular and fully featured
# implementation.

import tweepy

### Creating Twitter Credentials

For more in-depth instructions for creating a Twitter account and/or setting up a Twitter account to use the following code, I will provide a walkthrough on configuring and generating this information.

First, we assume you already have a Twitter account.
If this is not true, either create one real quick or follow along.
See the attached figures.

- __Step 1. Create a Twitter account__ If you haven't already done this, do this now at Twitter.com.

- __Step 2. Setting your mobile number__ Log into Twitter and go to "Settings." From there, click "Mobile" and fill in an SMS-enabled phone number. You will be asked to confirm this number once it's set, and you'll need to do so before you can create any apps for the next step.

<img src="files/TwitterInstructions_f1.png" scale="10%"/>
<img src="files/TwitterInstructions_f2.png" scale="10%"/>

- __Step 3. Create an app in Twitter's Dev site__ Go to (apps.twitter.com), and click the "Create New App" button. Fill in the "Name," "Description," and "Website" fields, leaving the callback one blank (we're not going to use it). Note that the website __must__ be a fully qualified URL, so it should look like: http://test.url.com. Then scroll down and read the developer agreement, checking that agree, and finally click "Create your Twitter application."

<img src="files/TwitterInstructions_f3.png" scale="10%"/>
<img src="files/TwitterInstructions_f4.png"/>

- __Step 4. Generate keys and tokens with this app__ After your application has been created, you will see a summary page like the one below. Click "Keys and Access Tokens" to view and manage keys. Scroll down and click "Create my access token." After a moment, your page should refresh, and it should show you four long strings of characters and numbers, a consume key, consumer secret, an access token, and an access secret (note these are __case-sensitive__!). Copy and past these four strings into the quotes in the code cell below.

<img src="files/TwitterInstructions_f5.png" scale="10%"/>
<img src="files/TwitterInstructions_f6.png"/>

In [6]:
# Use the strings from your Twitter app webpage to populate these four 
# variables. Be sure and put the strings BETWEEN the quotation marks
# to make it a valid Python string.

consumer_key = "pKkrahWC8Jn4WcJvUq7oFVX6v"
consumer_secret = "xEQQLEN2wYqY8aUTFLIseXzMx1RWiUDuVmxHTjvYZHAKILTVs7"
access_token = "363200844-POQQShs80z3dgULoNAZlFw8KsNbAMyVBIDT2YZPP"
access_secret = "qrAnYEun1uHVdIIdXk2CLV7PKN3OpcEdjeaejzimlVNac"

### Connecting to Twitter

Once we have the authentication details set, we can connect to Twitter using the Tweepy OAuth handler, as below.

In [85]:
# Now we use the configured authentication information to connect
# to Twitter's API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

print "Connected to Twitter!"

Connected to Twitter!


### Testing our Connection

Now that we are connected to Twitter, let's do a brief check that we can read tweets by pulling the first few tweets from our own timeline (or the account associated with your Twitter app) and printing them.

In [93]:
# Get tweets from our timeline
public_tweets = api.home_timeline()

# print the first five authors and tweet texts
for tweet in public_tweets[:5]:
    print tweet.author.screen_name, tweet.author.name, "said:", tweet.text

haldaume3 Hal Daumé III said: Congrats Graham! (Sorry to say Japanese food in Pgh, while surprisingly good, is going to be a step down for you :P) https://t.co/zdr24DVSrJ
haldaume3 Hal Daumé III said: RT @MSFTResearchCam: Would you like to intern with @katjahofmann and study how Minecraft can help provide equal opportunities? https://t.co…
haldaume3 Hal Daumé III said: RT @chaoticneural: Or "How to win citations and influence scientists"? https://t.co/LjpUDvIFbj
DistrictTrivia District Trivia said: #StudyUp with your #HintOfTheWeek for tonight's (5.25) #trivia! #EasyPoints https://t.co/7qig6YQH0O https://t.co/JnpgFgSKGW
jonfroehlich Jon Froehlich said: Congrats to #BodyVis team for earning a "Facilitator's Choice" award in the NSF Video Showcase: https://t.co/auYOqCYTg2! #stemvideohall


### Searching Twitter for Keywords

Now that we're connected, we can search Twitter for specific keywords with relative ease just like you were using Twitter's search box.
While this search only goes back 7 days and/or 1,500 tweets (whichever is less), it can be powerful if an event you want to track just started.

Note that you might have to deal with paging if you get lots of data. Twitter will only return you one page of up to 100 tweets at a time.

In [99]:
# Our search string
queryString = "earthquake"

# Perform the search
matchingTweets = api.search(queryString)

print "Searched for:", queryString
print "Number found:", len(matchingTweets)

# For each tweet that matches our query, print the author and text
print "\nTweets:"
for tweet in matchingTweets:
    print tweet.author.screen_name, tweet.text

Searched for: earthquake filter:media
Number found: 12

Tweets:
earthquakedisas 警戒！ [波形] 22:30 茨城県 大子 高萩、長野県 生坂 南木曽、北海道 赤井川 標茶北、岐阜県 谷汲、熊本県 益城、青森県 西目屋 (地中) #地震 #震度 #揺れ #earthquake https://t.co/mUCNMFHae3
ejx7tlyi1 [震源地] 熊本県熊本地方 [最大震度] 震度1 (2016年5月25日 22時07分頃発生) - goo 天気 https://t.co/YCT8eYRNVj | https://t.co/RGfnnGVDeb https://t.co/It2l2muLo8
kenyanwarden RT @KResearcher: The largest earthquake in recorded history to hit Kenya was on Jan 6 1928 in Subukia (6.9Ms). https://t.co/zW8kbJSZlV
earthquakedisas 警戒！ [波形] 22:29 茨城県 大子、長野県 生坂 南木曽、北海道 赤井川 標茶北、岐阜県 谷汲、熊本県 益城、青森県 西目屋 (地中) #地震 #震度 #揺れ #earthquake https://t.co/5sEvyVaXMy
earthquakedisas 警戒！ [波形] 22:25 茨城県 大子、長野県 生坂 南木曽、北海道 赤井川 標茶北、岐阜県 谷汲 揖斐川、青森県 西目屋 西目屋、熊本県 三角 益城 (地中) #地震 #震度 #揺れ #earthquake https://t.co/GiebV2Uqug
UtahsCW30 Did you feel the quake? Utah seismograph stations reporting a 4.1 mag earthquake hit about 1 a.m. near Duchesne. https://t.co/83P9JYGfsJ
abc4utah Did you feel the quake? Utah seismograph stations reporting a 4.1 mag

### More Complex Queries

Twitter's Search API exposes many capabilities, like filtering for media, links, mentions, geolocations, dates, etc.
We can access these capabilities directly with the search function.

For a list of operators Twitter supports, go here: https://dev.twitter.com/rest/public/search

In [100]:
# Lets find only media or links about earthquakes
queryString = "earthquake (filter:media OR filter:links)"

# Perform the search
matchingTweets = api.search(queryString)

print "Searched for:", queryString
print "Number found:", len(matchingTweets)

# For each tweet that matches our query, print the author and text
print "\nTweets:"
for tweet in matchingTweets:
    print tweet.author.screen_name, tweet.text

Searched for: earthquake (filter:media OR filter:links)
Number found: 12

Tweets:
IMC_UK We don't just respond We stay. One year on from the Nepal Earthquake, we're still there: https://t.co/ib9Y7mju5f https://t.co/jH6Vo2afWt
quakeredalert Forecast Earthquake confirmed https://t.co/mtyT7xofQR https://t.co/Yldrouw8YW
AroldoMaciel Forecast Earthquake confirmed https://t.co/BEiTw9SJ05 https://t.co/KIfNp61kno
AroldoMaciel An earthquake off the southeast coast of Crete, measuring 5.5 on the Richter scale, was recorded by the Institute... https://t.co/mvn5ztvcCq
beth_elliott123 RT @itsdjb: Trying to locate that earthquake  #ocrphysics https://t.co/94aiTLQxLP
OilnewsUa RT @CrudeOilPrices: Earthquake insurance latest concern in oil-rich Oklahoma https://t.co/0SFLdm2Oli via @UPI
IVEGABAJ2 #Sismo #Earthquake #Temblor  M 1.74, MONA PASSAGE                       https://t.co/fIJGm7lhzu  Puerto Rico
earthquakedisas 警戒！ [波形] 22:35 茨城県 大子 高萩 つくば、長野県 生坂 南木曽、北海道 赤井川 標茶北、岐阜県 谷汲、熊本県 益城、青森県 西目屋 (地中) #地震 #

### Dealing with Pages

As mentioned, Twitter serves results in pages. 
To get all results, we can use Tweepy's Cursor implementation, which handles this iteration through pages for us in the background.

In [103]:
# Lets find only media or links about earthquakes
queryString = "earthquake (filter:media OR filter:links)"

# How many tweets should we fetch? Upper limit is 1,500
maxToReturn = 100

# Perform the search, and for each tweet that matches our query, 
# print the author and text
print "\nTweets:"
for status in tweepy.Cursor(api.search, q=queryString).items(maxToReturn):
    print status.author.screen_name, status.text


Tweets:
Yifthael RT @eq_map: 【M4.4】VERACRUZ, MEXICO 141.6km 2016/05/25 21:24:06JST, 2016/05/25 12:24:06UTC
(G)https://t.co/qkWuZA5ymF (USGS)https://t.co/S0H…
preethamgupta @atikaCNN @SudhanshuTrived Media is insensitive to human emotions as proven during Nepal earthquake. #IBN7_Media420 https://t.co/PYqWsrLIL4
from___japan 【#USGS #Breaking】#earthquake　M 2.1 - 22km E of Anchorage, Alaska https://t.co/EtEn98h4GR #alert #tsunami #prayfromjapan
QuakeKitGuide Earthquake Myth - You can't plan for an Earthquake. That's totally WRONG! Read more...
https://t.co/q7dmq8K8A2 https://t.co/mA8EK7bSv0
Nandghar The buildings constructed under #ProjectNandghar are thermally insulated, #earthquake resistant &amp; dampness proof https://t.co/n4f6Hadinl
fusulinida2 RT @eq_map: 【M5.4】CRETE, GREECE 10.0km 2016/05/25 17:36:14JST, 2016/05/25 08:36:14UTC
(G)https://t.co/v5FPjDl8S5 (USGS)https://t.co/duXDIHI…
Frangotapia RT @AroldoMaciel: Forecast Earthquake confirmed https://t.co/BEiTw9SJ05 https://t.co/KIfNp

### Other Search Functionality

The Tweepy wrapper and Twitter API is pretty extensive.
You can do things like pull the last 3,200 tweets from other users' timelines, find all retweets of your account, get follower lists, search for users matching a query, etc.

More information on Tweepy's capabilities are available at its documentation page: (http://tweepy.readthedocs.io/en/v3.5.0/api.html)

Other information on the Twitter API is available here: (https://dev.twitter.com/rest/public/search).

### Twitter Streaming

Up to this point, all of our work has been retrospective. 
An event has occurred, and we want to see how Twitter responded over some period of time. 

To follow an event in real time, Twitter and Tweepy support Twitter streaming.
Streaming is a bit complicated, but it essentially lets of track a set of keywords, places, or users.

To keep things simple, I will provide a simple class and show methods for printing the first few tweets.
Larger solutions exist specifically for handling Twitter streaming.

You could take this code though and easily extend it by writing data to a file rather than the console.
I've marked where that code could be inserted.

In [166]:
# First, we need to create our own listener for the stream
# that will stop after a few tweets
class LocalStreamListener(tweepy.StreamListener):
    """A simple stream listener that breaks out after X tweets"""
    
    # Max number of tweets
    maxTweetCount = 10
    
    # Set current counter
    def __init__(self):
        tweepy.StreamListener.__init__(self)
        self.currentTweetCount = 0
    
    # Pass data up to parent then check if we should stop
    def on_data(self, data):

        print self.currentTweetCount
        
        tweepy.StreamListener.on_data(self, data)
            
        if ( self.currentTweetCount >= self.maxTweetCount ):
            return False

    # Increment the number of statuses we've seen
    def on_status(self, status):
        self.currentTweetCount += 1
        
        # Could write this status to a file instead of to the console
        print status.text
        
    # Error handling below here
    def on_exception(self, exc):
        print exc

    def on_limit(self, track):
        """Called when a limitation notice arrives"""
        print "Limit", track
        return

    def on_error(self, status_code):
        """Called when a non-200 status code is returned"""
        print "Error:", status_code
        return False

    def on_timeout(self):
        """Called when stream connection times out"""
        print "Timeout"
        return

    def on_disconnect(self, notice):
        """Called when twitter sends a disconnect notice
        """
        print "Disconnect:", notice
        return

    def on_warning(self, notice):
        print "Warning:", notice
        """Called when a disconnection warning message arrives"""



Now we set up the stream using the listener above

In [167]:
localStream = tweepy.Stream(api.auth, LocalStreamListener())

In [168]:
# Stream based on keywords
localStream.filter(track=['earthquake', 'disaster'])

0
RT @Be_YouClothing: I usually love @kfc UNTIL tonight! The store manager at Alpharetta store was VERY RUDE to my mother; his name is JAMES…
1
@CatarinaGirardi that tent was a recipe for disaster anyway then factor in rain and thunder... ill pass
2
SOLRIZE - Master of Disaster #BarbwiresRadio #Premier https://t.co/e4fJ61P57i #Rock
3
My kitchen may look like it was hit with a natural disaster, but I have made-from-scratch soft pretzels so I'm happy
4
RT @Max_Wolfe_: So this year was a disaster and I could not be happier that it's over
5
RT @JohnFromCranber: Is USA Being Set up For Disaster? RE: Iran Nuke Deal, illegals, EMP Attack, DOD Cuts, Islamization, Fed Debt #tcot htt…
6
The other thing: I'm not convinced there's a single thing on @AceofSpadesHQ's disaster list that Trump wouldn't do https://t.co/vdWobuyv09
7
Moderate earthquake - Chichi-shima, Japan on May 25, 2016 https://t.co/04CzarW3Nq via Earthquake-Report
8
@KomariKills What A Disaster
9
RT @AmSciMag: Gauging #earthquake ha

In [130]:
# List of screen names to track
screenNames = ['bbc', 'itele']

# Twitter stream uses user IDs instead of names
# so we must convert
userIds = []
for sn in screenNames:
    user = api.get_user(sn)
    userIds.append(user.id_str)

# Stream based on users
localStream.filter(follow=userIds)

Error: 420


In [ ]:
# Specify coordinates for a bounding box around area of interest
# In this case, we use San Francisco
swCornerLat = 36.8
swCornerLon = -122.75
neCornerLat = 37.8
neCornerLon = -121.75

boxArray = [swCornerLon, swCornerLat, neCornerLon, neCornerLat]

# Stream based on location
localStream.filter(locations=boxArray)